# Import tools

In [ ]:
import numpy as np
import torch
import h5py

# Using spectral methods to compute the ground truth solution of the PDE
Poisson function is given by:
$-div\ K \nabla u = f$

Helmholtz function is given by:
$- \nabla^2 U + \omega u = f$


Advection-Diffusion function is given by:
$- div\ K\nabla u + v.\nabla u = f$

## Spectral Method with Radial Gaussian Basis Function
Resolution of the grid is $128 \times 128$.\
The given expansion used:\
$$\begin{equation}
u(x) ≈ u^{(N)}(x) = \sum^{N}_{m=1} \phi_i(x)p_i;\ \ \phi_i(x) = \phi(x-x_i)\\
\end{equation}
$$
Here $\phi(x)$ is gaussian with standard diviation $\sigma = \frac{1}{32}$ i.e $4$ times the spatial resolution. Spacing between gaussians is $2\sigma$. $s$ fraction of the $p_i$ value are set to zero.

In [ ]:
ntrain = 100 # number of training examples
nval = 25 # number of validation examples
ntest = 25 # number of testing examples

nx = ny = 128 # grid size nxn
lx = ly = 1
dx = lx/nx
dy = ly/ny

std = 1/32
space = 2*std
vfs = [0.2, 0.4, 0.6, 0.8]
ng = 144 # number of gaussians in grid
o1 = 1 # sample wavenumbers starting from o1
o2 = 10 # sample wavenumbers starting from o2
diff_coef_scale = 0.01


x = np.arange(0, lx, dx)
y = np.arange(0, ly, dy)

In [ ]:
vf = vfs[0]
u, source, om = helmholtz(x, y, std, space, nx, ny, lx, ly, vf, ng, o1, o2, diff_coef_scale)

In [67]:
import numpy as np


x = np.arange(0, 5, 1)
y = np.arange(0, 6, 1)
x_cord, y_cord = np.meshgrid(x, y)


centers_x = np.arange(-2, 2, 1)
centers_y = np.arange(-2, 3, 1)

# Create meshgrid from centers_x and centers_y
centers_x_grid, centers_y_grid = np.meshgrid(centers_x, centers_y)

p = np.where((0.999 *np.random.rand(*centers_x_grid.shape) + 0.001) < 0.5, 0,
             np.random.rand(*centers_x_grid.shape))

# Reshape p to have a shape compatible with broadcasting with grid
p_reshaped = p.reshape(1, 1, *p.shape)

# Calculate distances using vectorized operations
grid = (x_cord[..., np.newaxis, np.newaxis] - centers_x_grid)**2 + (y_cord[..., np.newaxis, np.newaxis] - centers_y_grid)**2

u_grid = np.sum(grid * p_reshaped, axis=(2, 3))

In [68]:
x_cord
y_cord
centers_x_grid
centers_y_grid
centers_x_grid.shape
grid
p
p_reshaped
u_grid
u_grid/np.max(u_grid)

array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]])

In [ ]:
class Grid:
    def __init__(self, nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True, isRemoval_Intregal =True):
        self.nx = nx
        self.ny = ny
        self.Lx = Lx
        self.Ly = Ly
        self.std = std
        self.space = 2 * self.std
        self.dx = self.Lx / self.nx
        self.dy = self.Ly / self.ny
        self.isNormalize = isNormalize
        self.num_gaussians = num_gaussians
        self.isRemoval_Intregal = isRemoval_Intregal

        x = np.arange(0, self.Lx, self.dx)
        y = np.arange(0, self.Ly, self.dy)
        self.x_cord, self.y_cord = np.meshgrid(x, y)

    def rbf(self, center=(0.5, 0.5)):
        # Implementation of radial basis functions
        num_centers_sqrt = np.sqrt(self.num_gaussians)  # Number of centers in each direction
        grid_length = (num_centers_sqrt - 1) * self.space  # Length of the grid in each direction

        # Generate grid of centers
        centers_x = np.arange(center[0] - grid_length / 2, center[0] + grid_length / 2 + spacing, self.space)
        centers_y = np.arange(center[1] - grid_length / 2, center[1] + grid_length / 2 + spacing, self.space)

        # Create meshgrid from centers_x and centers_y
        centers_x_grid, centers_y_grid = np.meshgrid(centers_x, centers_y)

        # Calculate distances using vectorized operations
        grid_distance_squared = ((x_cord[..., np.newaxis, np.newaxis] - centers_x_grid)**2 + (y_cord[..., np.newaxis, np.newaxis] - centers_y_grid)**2)

        # Generating a random array and zeroing out x percent of its elements
        p = np.where((0.999 * np.random.rand(*centers_x_grid.shape)+ 0.001) < vf, 0,
                     np.random.rand(*centers_x_grid.shape))

        # Reshape p to have a shape compatible with broadcasting with grid
        p_reshaped = p.reshape(1, 1, *p.shape)

        # Calulate the source function
        R = 2 * sigma**2
        u_basis_form = np.exp(grid_distance_squared/R)
        u_grid = u_basis_form * p_reshaped

        # Normalize the basis functions
        if self.isNormalize:
            u_grid /= np.max(u_grid)

        # Remove integral of the basis function
        if self.isRemoval_Intregal:
            integral_mean = np.mean(u_grid.flatten())
            u_grid -= integral_mean

        return u_grid

    def fft_coef(self, u):
        # Implementation of FFT coefficients
        nx, ny = u.shape

        if not is_torch:
            ikx = 1j * np.concatenate((np.arange(0, nx//2+1, 1),
                                       np.arange(-nx//2+1, 0, 1)))
            iky = 1j * np.concatenate((np.arange(0, ny//2+1, 1),
                                       np.arange(-ny//2+1, 0, 1)))

            ikx = ikx.reshape(1, nx).repeat(ny, axis=0)
            iky = iky.reshape(ny, 1).repeat(nx, axis=1)
        else:
            ikx = 1j * torch.cat((torch.arange(0, nx/2+1, 1),
                                  torch.arange(-nx/2+1, 0, 1)))
            iky = 1j * torch.cat((torch.arange(0, ny/2+1, 1),
                                  torch.arange(-ny/2+1, 0, 1)))

            ikx = np.repeat(ikx.reshape(1, -1), ny, axis=0)
            iky = np.repeat(iky.reshape(-1, 1), nx, axis=1)

        return ikx, iky

    def laplacian(self, u):
        # Implementation of laplacian operator
        ikx, iky = self.fft_coef(u)
        ikx2 = ikx**2
        iky2 = iky**2

        u_hat = np.fft.fft2(u)
        u_hat *= (ikx2+iky2) * (4.0 * np.pi**2)/(self.Lx*self.Ly)

        return np.real(np.fft.ifft2(u_hat))

    def div(self, u):
        # Implementation of divergence operator
        ikx, iky = self.fft_coef(u)
        ux_hat = np.fft.fft2(ux)
        uy_hat = np.fft.fft2(uy)

        u1 = np.real(np.fft.ifft2(ux_hat * ikx)) * (2.0 * np.pi) / self.Lx
        u2 = np.real(np.fft.ifft2(uy_hat * iky)) * (2.0 * np.pi) / self.Ly

        return u1 + u2

In [ ]:
class EquationSolver(Grid):
    def __init__(self, equation_type='helmholtz', num_gaussians=144, grid_size=128, std=1/32, Lx=1, Ly=1, sparse=False, data_path="./"):
        super().__init__(grid_size=128, std=1/32, Lx=1, Ly=1)
        self.equation_type = equation_type
        self.num_gaussians = num_gaussians
        self.sparse = sparse
        self.data_path = data_path

    def helmholtz(self):
        # Implementation for solving Helmholtz equation using Grid methods

        u_grid = self.rbf()

        omega = np.random.randint(self.o1, self.o2+1)
        self.laplacian(u)

        u_grid_hat = np.fft.fft2(u_grid)






        return u, source, omega

    def poisson(self):
        # Implementation for solving Poisson equation using Grid methods
        return u, source, k

    def advection_diffusion(self):
        # Implementation for solving Advection-Diffusion equation using Grid methods
        return u, source, diffusion_tensor

    def create_hdf5(self, path, dat, ten):
        # Implementation to create HDF5 files
        pass

    def generate_data(self, ntrain=100, nval=25, ntest=25, e1=1, e2=5, o1=1, o2=10):
        train = np.zeros((ntrain, 2, self.grid_size, self.grid_size))
        train_info = np.zeros((ntrain, 2))
        val = np.zeros((nval, 2, self.grid_size, self.grid_size))
        val_info = np.zeros((nval, 2))
        test = np.zeros((ntest, 2, self.grid_size, self.grid_size))
        test_info = np.zeros((ntest, 2))

        for idx, vf in enumerate([0.2, 0.4, 0.6, 0.8]):
            if self.equation_type == 'helmholtz':
                u, source, info = self.helmholtz()
            elif self.equation_type == 'poisson':
                u, source, info = self.poisson()
            elif self.equation_type == 'advdiff':
                u, source, info = self.advection_diffusion()

            # Assign data to appropriate datasets (train, validation, test)

        # Save data into HDF5 files
